In [1]:
import tensorflow.keras as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
import tf2onnx
import numpy as np
import math

In [2]:
batch_size = 500 # Number of images processed at once
nb_classes = 10  # 10 Digits from 0 to 9

# Dimensionen of the input images (28x28 pixel)
img_rows, img_cols = 28, 28

# Load image data with labels, split into test and training set 
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# reshape images in 4D tensor (N images, 28 rows, 28 columns, 1 channel) 
# rescale pixels range from [0, 255] to [0, 1]
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
print('X_train shape: ', X_train.shape)
print(X_train.shape[0], "training samples")
print(X_test.shape[0], "test samples")

# convert digit labels (0-9) in one-hot encoded binary vectors. 
# These correspond to the training/test labels at the output of the net. 
Y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
Y_test = tf.keras.utils.to_categorical(y_test, nb_classes)
print("One-hot encoding: {}".format(Y_train[0, :]))

X_train shape:  (60000, 28, 28, 1)
60000 training samples
10000 test samples
One-hot encoding: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() ## for Lenet5

In [4]:
class DataGenerator(tf.compat.v2.keras.utils.Sequence):
 
    def __init__(self, X_data , y_data, batch_size, dim, n_classes,
                 to_fit, shuffle = True):
        self.batch_size = batch_size
        self.X_data = X_data
        self.labels = y_data
        self.y_data = y_data
        self.to_fit = to_fit
        self.n_classes = n_classes
        self.dim = dim
        self.shuffle = shuffle
        self.n = 0
        self.list_IDs = np.arange(len(self.X_data))
        self.on_epoch_end()
    def __next__(self):
        # Get one batch of data
        data = self.__getitem__(self.n)
        # Batch index
        self.n += 1
        
        # If we have processed the entire dataset then
        if self.n >= self.__len__():
            self.on_epoch_end
            self.n = 0
        
        return data
    def __len__(self):
        # Return the number of batches of the dataset
        return math.ceil(len(self.indexes)/self.batch_size)
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:
            (index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X = self._generate_x(list_IDs_temp)
        
        if self.to_fit:
            y = self._generate_y(list_IDs_temp)
            return X, y
        else:
            return X
    def on_epoch_end(self):
        
        self.indexes = np.arange(len(self.X_data))
        
        if self.shuffle: 
            np.random.shuffle(self.indexes)
    def _generate_x(self, list_IDs_temp):
               
        X = np.empty((self.batch_size, *self.dim))
        
        for i, ID in enumerate(list_IDs_temp):
            
            X[i,] = self.X_data[ID]
            
            # Normalize data
            X = (X/255).astype('float32')
            
        return X[:,:,:, np.newaxis]
    def _generate_y(self, list_IDs_temp):
        
        y = np.empty(self.batch_size)
        
        for i, ID in enumerate(list_IDs_temp):
            
            y[i] = self.y_data[ID]
            
        return keras.utils.to_categorical(y,num_classes=self.n_classes)

In [5]:
n_classes = 10
nb_classes = 10
input_shape = (28, 28) #Lenet5

In [6]:
training_id = 1
model_short_name = 'Lenet5'
framework = 'Keras'

In [7]:
train_generator = DataGenerator(X_train, Y_train, batch_size = batch_size,
                                dim = input_shape,
                                n_classes=nb_classes, 
                                to_fit=True, shuffle=True)
val_generator =  DataGenerator(X_test, Y_test, batch_size=batch_size, 
                               dim = input_shape, 
                               n_classes= nb_classes, 
                               to_fit=True, shuffle=True)

In [8]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/Keras/lenet5/'
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
model_name = 'tf_Lenet5_mnist_2021-10-27_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

601864

In [9]:
size1 = os.path.getsize(path+ model_name+'.h5')
size1

601864

In [10]:
import onnx
import onnxruntime
import coremltools
import time

Using TensorFlow backend.


In [11]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
restored_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/restored/'

In [12]:
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/errors/'

In [15]:
def to_onnx(i, x, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    with tf.device('/cpu:0'): 
        k_predict = model.predict(x)
    t_elapsed_1 = time.time() - since_1
    # Export the model
    since_1 = time.time()
    
    since_onnx = time.time()
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model,
                input_signature=None, opset=11, custom_ops=None,
                custom_op_handlers=None, custom_rewriter=None,
                inputs_as_nchw=None, extra_opset=None, shape_override=None,
                 target=None, large_model=False, output_path=onnx_path+'keras/{}.onnx'.format(model_name))
    
    t_elapsed_2 = time.time() - since_1
    
    onnx_model = onnx.load(onnx_path+"keras/{}.onnx".format(model_name))
    onnx.checker.check_model(onnx_model)
    size2 = os.path.getsize(onnx_path+"keras/{}.onnx".format(model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+"keras/{}.onnx".format(model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    t_elapsed_3 = time.time() - since_1
    t_elapsed_ = time.time() - since_
    t_elapsed_onnx = time.time() - since_onnx
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    abs_err = np.absolute(k_predict-ort_outs[0])
    rel_err = np.absolute(k_predict-ort_outs[0])/ np.absolute(ort_outs[0])
    #print('Batch: ', i, abs_err, rel_err)
    

    
    ## Converting the coremltool
    since_1 = time.time()
    coreml_model = coremltools.convert(model)
    t_elapsed_4 = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+'keras/{}.mlmodel'.format(model_name))
    t_elapsed_5 = time.time() - since_1
    
    size3 = os.path.getsize(coreml_path+'keras/{}.mlmodel'.format(model_name))
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({'conv2d_input':x})
    t_elapsed_6 = time.time() - since_1
    t_elapsed2_ = time.time() - since_
    
    abs_err2 = np.absolute(k_predict-output_dict_test['Identity'])
    rel_err2 = np.absolute(k_predict-output_dict_test['Identity'])/ np.absolute(output_dict_test['Identity'])
    
    for j in range (len(abs_err)):
        for k in range(len(abs_err[j])): 
            data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])
    
    
    for j in range (len(abs_err2)):
        for k in range(len(abs_err2[j])): 
            data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])
      
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])

In [16]:
def _lets_convert(data, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    since = time.time()
    for i, (images, labels) in enumerate(data):
        #torch.cuda.empty_cache()
        #images = images.cuda()
        to_onnx(i, images,data_writer_error, data_writer_error2, data_writer_run, batch_size)
        if i == 20:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    
    #return list_converted
    #return list_converted, abs_errors, rel_errors, 'Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), np.mean(np.array(t0_list)), np.mean(np.array(t1_list)), np.mean(np.array(t2_list)), np.mean(np.array(t3_list)), np.mean(np.array(s_list))

In [17]:
#model_name = 'letnet5-keras'
import pandas as pd 
data_file_error = open(error_path + 'keras/onnx_error_metrics_{}.csv'.format(model_name), mode='w', newline='',
                                  encoding='utf-8')
data_writer_error = csv.writer(data_file_error, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
data_writer_error.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


data_file_error2 = open(error_path + 'keras/mlmore_error_metrics_{}.csv'.format(model_name), mode='w', newline='',
                                  encoding='utf-8')
data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


data_file_run = open(error_path + 'keras/runtime_metrics_{}.csv'.format(model_name), mode='w', newline='',
                                  encoding='utf-8')
data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])

for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,128, 150,200, 250,300,350, 400, 450, 500]:
    print("################ Batch size: ", batch_size)
    val_generator =  DataGenerator(x_test, y_test, batch_size=batch_size,  dim = input_shape, 
                               n_classes= nb_classes, 
                               to_fit=True, shuffle=True)
    _lets_convert(val_generator, data_writer_error, data_writer_error2, data_writer_run, batch_size)
data_file_error.close()
data_file_run.close()

################ Batch size:  1
converting for batch:  0

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 54/54 [00:00<00:00, 1400.19 ops/s]


converting for batch:  1

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 54/54 [00:00<00:00, 1428.88 ops/s]


converting for batch:  2

*********




...

converting for batch:  18

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 54/54 [00:00<00:00, 1486.01 ops/s]


converting for batch:  19

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 54/54 [00:00<00:00, 1571.69 ops/s]


Conversion complete in 0m 44s


# Tests

In [16]:
## Lets first load just one batch of the dataset for testing

In [18]:
val_generator =  DataGenerator(x_test, y_test, batch_size=1,  dim = input_shape, 
                               n_classes= nb_classes, 
                               to_fit=True, shuffle=True)
for i, (images, labels) in enumerate(val_generator):
    x, y = images, labels
    break

## Converting pytorch model to onnx then to Keras

In [35]:
## Convert to onnx runtime
model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model, input_names=['inputs'],
                input_signature=None, opset=11, custom_ops=None,
                custom_op_handlers=None, custom_rewriter=None,
                inputs_as_nchw=None, extra_opset=None, shape_override=None,
                 target=None, large_model=False, output_path=onnx_path+'keras/lenet5/{}.onnx'.format(model_name))

TypeError: from_keras() got an unexpected keyword argument 'input_names'

In [17]:
from onnx2keras import onnx_to_keras
from onnx_tf.backend import prepare

In [22]:
onnx_model = onnx.load(onnx_path+'keras/lenet5/{}.onnx'.format(model_name))
tf_rep = prepare(onnx_model)  # prepare tf representation
print(tf_rep.inputs)
#k_model = onnx_to_keras(onnx_model, tf_rep.inputs)
tf_rep.export_graph(restored_path+'keras/lenet5/{}.pb'.format(model_name))

['conv2d_input']


In [23]:
with tf.gfile.FastGFile(restored_path+'keras/lenet5/{}.pb'.format(model_name), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [28]:
## scikit-learn 0.19.2 should be okay
#!pip install scikit-learn==0.19.2

## Lets try to restore the model Here...

In [14]:
onnx_model = onnx.load(onnx_path+"keras/{}/{}.onnx".format(model_short_name, model_name))
onnx.checker.check_model(onnx_model)
size2 = os.path.getsize(onnx_path+"keras/{}/{}.onnx".format(model_short_name,model_name))

In [15]:
from onnx2keras import onnx_to_keras

In [16]:
k_model_restored = onnx_to_keras(onnx_model, ['input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> conv2d_input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> dense_1.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight sequential/dense_1/MatMul/ReadVariableOp:0 with shape (84, 10).
DEBUG:onnx2keras:Found weight sequential/dense_1/BiasAdd/ReadVariableOp:0 with shape (10,).
DEBUG:onnx2keras:Found weight sequential/dense/MatMul/ReadVariableOp:0 with shape (1080, 84).
DEBUG:onnx2keras:Found weight sequential/dense/BiasAdd/ReadVariableOp:0 with shape (84,).
DEBUG:onnx2keras:Found weight sequential/conv2d_2/Conv2D/ReadVariableOp:0 with shape (120, 16, 5, 5).
DEBUG:onnx2keras:Found weight sequential/conv2d_2/BiasAdd/ReadVariableOp:0 with shape (120,).
DEBUG:onnx2keras:Found weight sequential/conv2d_1/Conv2D/ReadVariableOp:0 with shape (16, 6, 5, 5).
DEBUG:onnx2keras:Found weight sequential/conv2d_

AttributeError: Current node is not in weights / model inputs / layers.

## Let's try to convert this model to coreml

In [23]:
# install it first
#!pip install coremltools
'''WARNING:root:scikit-learn version 0.21.3 is not supported. Minimum required version: 0.17. Maximum required version: 0.19.2. Disabling scikit-learn conversion API.
WARNING:root:TensorFlow version 2.5.0 detected. Last version known to be fully compatible is 2.3.1 .
Using TensorFlow backend.'''

In [17]:
import coremltools

Using TensorFlow backend.


In [18]:
coreml_model = coremltools.convert(model)

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 54/54 [00:00<00:00, 1511.08 ops/s]


In [33]:
coreml_model2 = coremltools.convert(model, input_name='input_name', image_input_name='input_names')

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 54/54 [00:00<00:00, 1544.94 ops/s]


name: "conv2d_input"
type {
  multiArrayType {
    shape: 1
    shape: 28
    shape: 28
    shape: 1
    dataType: FLOAT32
    shapeRange {
      sizeRanges {
        lowerBound: 1
        upperBound: -1
      }
      sizeRanges {
        lowerBound: 28
        upperBound: 28
      }
      sizeRanges {
        lowerBound: 28
        upperBound: 28
      }
      sizeRanges {
        lowerBound: 1
        upperBound: 1
      }
    }
  }
}

In [19]:
print(coreml_model.get_spec().description.input)

[name: "conv2d_input"
type {
  multiArrayType {
    shape: 1
    shape: 28
    shape: 28
    shape: 1
    dataType: FLOAT32
    shapeRange {
      sizeRanges {
        lowerBound: 1
        upperBound: -1
      }
      sizeRanges {
        lowerBound: 28
        upperBound: 28
      }
      sizeRanges {
        lowerBound: 28
        upperBound: 28
      }
      sizeRanges {
        lowerBound: 1
        upperBound: 1
      }
    }
  }
}
]


In [20]:
#input_shape = (1,28,28)
output_dict_test = coreml_model.predict({'conv2d_input':X_test})

In [21]:
output_dict_test['Identity']

array([[3.3256106e-21, 6.8065322e-19, 6.5434598e-18, ..., 1.0000000e+00,
        1.7159787e-19, 1.8412391e-16],
       [8.1228532e-02, 1.3294430e-01, 8.4695749e-02, ..., 1.0460241e-01,
        1.5361437e-01, 9.5107198e-02],
       [8.1228532e-02, 1.3294430e-01, 8.4695749e-02, ..., 1.0460241e-01,
        1.5361437e-01, 9.5107198e-02],
       ...,
       [8.1228532e-02, 1.3294430e-01, 8.4695749e-02, ..., 1.0460241e-01,
        1.5361437e-01, 9.5107183e-02],
       [8.1228532e-02, 1.3294430e-01, 8.4695749e-02, ..., 1.0460241e-01,
        1.5361437e-01, 9.5107183e-02],
       [8.1228532e-02, 1.3294430e-01, 8.4695749e-02, ..., 1.0460241e-01,
        1.5361437e-01, 9.5107183e-02]], dtype=float32)

In [22]:
Y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
np.testing.assert_array_equal(Y_test, output_dict_test['Identity'])

AssertionError: 
Arrays are not equal

Mismatched elements: 99999 / 100000 (100%)
Max absolute difference: 0.9318618
Max relative difference: 13.676064
 x: array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],...
 y: array([[3.325611e-21, 6.806532e-19, 6.543460e-18, ..., 1.000000e+00,
        1.715979e-19, 1.841239e-16],
       [8.122853e-02, 1.329443e-01, 8.469575e-02, ..., 1.046024e-01,...

In [24]:
batch_size = 1

val_generator =  DataGenerator(x_test, y_test, batch_size=batch_size,  dim = input_shape, 
                               n_classes= nb_classes, 
                               to_fit=True, shuffle=True)

In [26]:
for i, (images, labels) in enumerate(val_generator): 
    k_predict = model.predict(images)
    output_dict_test = coreml_model.predict({'conv2d_input':images})
    #print()
    #np.testing.assert_array_equal(Y_test, output_dict_test['Identity'])
    break

In [27]:
k_predict

array([[1.0000000e+00, 1.3026834e-13, 2.7554650e-11, 4.7366053e-15,
        3.2157813e-14, 5.1787448e-13, 8.5086306e-11, 4.3897487e-13,
        5.8446173e-13, 3.7581365e-11]], dtype=float32)

In [28]:
output_dict_test['Identity']

array([[1.0000000e+00, 1.3553026e-13, 2.8510128e-11, 4.9403252e-15,
        3.3564270e-14, 5.3701808e-13, 8.7776848e-11, 4.5606216e-13,
        6.0698175e-13, 3.8738769e-11]], dtype=float32)

In [40]:
abs_err2 = np.absolute(k_predict-output_dict_test['Identity'])
rel_err2 = np.absolute(k_predict-output_dict_test['Identity'])/ np.absolute(output_dict_test['Identity'])

In [41]:
print(abs_err2, rel_err2)

[[0.0000000e+00 5.2619177e-15 9.5547875e-13 2.0371989e-16 1.4064576e-15
  1.9143595e-14 2.6905422e-12 1.7087297e-14 2.2520018e-14 1.1574040e-12]] [[0.         0.03882467 0.03351366 0.04123613 0.04190341 0.03564795
  0.03065207 0.03746704 0.03710164 0.02987715]]


In [29]:
np.testing.assert_array_equal(k_predict, output_dict_test['Identity'])

AssertionError: 
Arrays are not equal

Mismatched elements: 9 / 10 (90%)
Max absolute difference: 2.6905422e-12
Max relative difference: 0.04190341
 x: array([[1.000000e+00, 1.302683e-13, 2.755465e-11, 4.736605e-15,
        3.215781e-14, 5.178745e-13, 8.508631e-11, 4.389749e-13,
        5.844617e-13, 3.758137e-11]], dtype=float32)
 y: array([[1.000000e+00, 1.355303e-13, 2.851013e-11, 4.940325e-15,
        3.356427e-14, 5.370181e-13, 8.777685e-11, 4.560622e-13,
        6.069818e-13, 3.873877e-11]], dtype=float32)